In [100]:
from scrape_data import get_runner_ids, get_splits, load_nyc_data
import pandas as pd
# import requests
# import json

# x = get_runner_ids(eventCode="M2023", place1=51340, place2=51348)
# get_splits(runner_id=43928778)  # AssertionError

yr = 19
load_info = {24: 55270, 23: 51348, 22: 47745, 21: 24949, 19: 53520, 18: 52706, 17: 50641, 16: 51275}
num = load_info[yr]
# tables = load_nyc_data(last_runner=num, eventCode=f"M20{yr}")
# pd.concat(tables).to_csv(f"raw_data/nyc/nyc_names{yr}.csv", index=False)

In [ ]:
# pd.concat(tables).to_csv("raw_data/nyc/nyc_names19.csv", index=False)

In [97]:
combined = pd.concat([pd.read_csv("raw_data/nyc/nyc_times23.csv"), pd.read_csv("raw_data/nyc/nyc_times23_0.csv"), pd.read_csv("raw_data/nyc/nyc_times23_1.csv"), pd.read_csv("raw_data/nyc/nyc_times23_.csv")])
combined.drop("Unnamed: 0", axis=1).drop_duplicates().to_csv("raw_data/nyc/nyc_times23.csv", index=False)
# combined.drop("Unnamed: 0", axis=1).drop_duplicates()

In [ ]:
# pd.read_csv(f"raw_data/nyc/nyc_names{year}.csv").merge(combined.drop("Unnamed: 0", axis=1).drop_duplicates(), left_on="runnerId", right_on="id")
year=23
d_load = pd.read_csv(f"raw_data/nyc/nyc_names{year}.csv")
times = pd.read_csv(f"raw_data/nyc/nyc_times{year}.csv")
d_load.merge(times, left_on="runnerId", right_on="id").to_csv(f"raw_data/nyc/nyc{year}.csv")

Chicago

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
def scrape_chi_data(yr=23):
    event_param = {23: "MAR_9TGG963812D", 22: "MAR_9TGG9638119", 21: "MAR_9TGG9638F1", 19: "MAR_999999107FA31100000000C9"}
    links_list = pd.read_csv(f"raw_data/chicago/ids{year}.csv")['0'].values

    results = []
    for idx, person in enumerate(links_list[:100]):
        params = {"content" : "detail", "idp" : person, "lang" : "EN", "event" : event_param[year],}
        response = requests.post("https://chicago-history.r.mikatiming.com/2023", params=params)
        soup = BeautifulSoup(response.text, 'lxml')
        table = soup.find("table", attrs={"class":"table table-condensed table-striped"})
        rows = table.findAll("tr")
        splits = [[t.text.strip() for t in row.findAll(["th", "td"])][2] for row in rows[1:]]
        rows2 = soup.findAll(name="table", attrs={"class":"table table-condensed"})[1]#.findAll("th")
        info = [row.text.strip() for row in rows2.findAll(["th", "td"])][1::2]
        results.append(info + splits)
        print(idx)

    mks = [[t.text.strip() for t in row.findAll(["th", "td"])][0] for row in rows[1:]]
    cols = [row.text.strip() for row in rows2.findAll(["th", "td"])][::2]
    pd.DataFrame(results, columns = cols + mks).to_csv(f"raw_data/chicago/chi{yr}.csv")


In [ ]:
pd.DataFrame(results, columns = cols + mks).to_csv("")

In [ ]:
event_ = {23: "MAR_9TGG963812D", 22: "MAR_9TGG9638119", 21: "MAR_9TGG9638F1", 19: "MAR_999999107FA31100000000C9"}

# params = {
#     "content" : "detail",
#     # "fpid" : "search",
#     # "pid" : "search",
#     "idp" : "9TGG9638284476",
#     "lang" : "EN",
#     "event" : "MAR_9TGG963812D",
#     # "lang" : "EN_CAP",
#     # "search_event" : "ALL_EVENT_GROUP_2022",
# }

params = {"content" : "detail", "idp" : "9TGG9638284476", "lang" : "EN", "event" : "MAR_9TGG963812D",}
response = requests.post("https://chicago-history.r.mikatiming.com/2023", params=params)
soup = BeautifulSoup(response.text, 'lxml')
table = soup.find("table", attrs={"class":"table table-condensed table-striped"})
rows = table.findAll("tr")
splits = [[t.text.strip() for t in row.findAll(["th", "td"])][2] for row in rows[1:]]
splits

In [ ]:
pd.DataFrame(splits_list, columns=[[t.text.strip() for t in row.findAll(["th", "td"])][0] for row in rows[1:]])
